## Select a MSQ Tide file

In [ ]:
using CSV
using Dates, DataFrames, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics
using Tk

################################################
################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# Pick directory containing MSQ .csv files
msq_directory = pick_folder()

# build list of all msq files in selected directory
msq_files = filter(x->occursin("_10min",x), readdir(msq_directory));
##msq_files = filter(x->occursin("_hourly",x), readdir(msq_directory));
msq_files = msq_files[findall(x->endswith(uppercase(x), ".CSV"), msq_files)]

# Check whether any msq files exist in selected directory. If not, EXIT!
if length(msq_files) == 0
    println("No msq files found in "*msq_directory)
    exit;
else
    println(string(length(msq_files)) * " MSQ tide files found")
end

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, msq_files)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    global file_choice = get_value(lb);
    
    # Select a msq file
    global infil = msq_directory * "\\" * file_choice[1]
    
    global title = file_choice[1]
    
    println("Selected ",infil)

    global msq_df = DataFrame(CSV.File(infil; header=false, skipto=41, delim=", ", ignorerepeated=true));
    msq_df.Date = DateTime.(Date.(strip.(msq_df.Column1),"dd/mm/yyyy"),Time.(strip.(msq_df.Column2),"HH:MM"))      
    rename!(msq_df,[:Column3,:Column4] .=> [:Ind, :Reading]);
    select!(msq_df, [:Date, :Ind, :Reading], Not([:Column1, :Column2]));
    msq_df.Reading = parse.(Float64,msq_df.Reading)
    
    # Convert -9999 values to Nans for plotting
    msq_df.Reading[findall(abs.(msq_df.Reading).>9)] .= NaN
    
    msq_df.time_diff = [missing; diff(msq_df.Date)]
    
    global first_date = first(msq_df.Date)
    global last_date = last(msq_df.Date)
    
    year_diff = (Year(last_date) |> Dates.value) - (Year(first_date) |> Dates.value)
    
    # set X-axis scale based on amount of data
    if (year_diff == 0)
        mth_val = 1
    elseif (year_diff == 1)
        mth_val = 2
    elseif (year_diff == 2)
        mth_val = 3
    else
        mth_val = 6
    end
    
    # display plots to screen
    tm_tick = range(first_date,last_date,step=Month(mth_val))
    ticks = Dates.format.(tm_tick,"YY-mm")
    
    tides_plot = plot(msq_df.Date,msq_df.Reading, size = (1400, 600), label="",
##        marker = :circle, markersize = 1,
        xlims=(first_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=7,ytickfontsize=8, 
        framestyle = :box, legend=:bottomleft, title=title, 
        margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)
    
    display(tides_plot)
    
    start_date = first_date
    println(title,"\n")
    println("Mth Year  Gaps     Good     Minimum     Maximum     Mean      Range    St Devn")

    total_gaps = 0
    total_good = 0

    yearly_mean = 0
    no_of_months = 0
    
    global yearly_averages_df = DataFrame([[],[]], ["Year", "Mean_WL"])
    
    while start_date <= last_date
        
        # select a month of data and store month of data in temporary df
        end_date = start_date + Dates.Month(1)
        month_df = msq_df[findall(start_date .<= msq_df.Date .< end_date),:]

        # determine total number of 10-minute records possible for the month
        total_possible = trunc(Int,(end_date - start_date) / Millisecond(1) * (1 / 600000))
        total_actual = nrow(month_df)
        
        # determine the number of gaps
        gaps = total_possible - total_actual

        # get totals of gaps and good data
        total_gaps = total_gaps + gaps
        total_good = total_good + total_actual

        # get monthly values
        month = Month(start_date) |> Dates.value
        year = Year(start_date) |> Dates.value
        
        if nrow(month_df) > 1
            monthly_min = minimum(month_df.Reading)
            monthly_max = maximum(month_df.Reading)
            monthly_mean = mean(month_df.Reading)
            monthly_std = std(month_df.Reading)
            monthly_range = monthly_max - monthly_min

        else
            monthly_min = 0
            monthly_max = 0
            monthly_mean = 0
            monthly_std = 0
            monthly_range = 0
        end

        @printf("%3i %4i %5i %8i %10.3f %11.3f %10.3f %9.3f %9.3f\n", month, year, gaps, total_actual, monthly_min, monthly_max, monthly_mean, monthly_range, monthly_std)
        
        # add monthly mean to yearly total, and increment number of months used
        yearly_mean = yearly_mean + monthly_mean
        no_of_months = no_of_months + 1

        # print annual averages for current year of data
        if (Year(end_date) |> Dates.value) != (Year(start_date) |> Dates.value)
            
            year_valu = Year(start_date) |> Dates.value
            year_total = mean(msq_df[findall(Year.(msq_df.Date) .|> Dates.value .== year),:].Reading)
            # add yearly average to df
            push!(yearly_averages_df,[year_valu,year_total])
            
            println("=============================================================================")
            @printf("%s %4i %s %5.3f\n","                         Yearly average for",(year_valu),"= ", year_total)
            @printf("%s%5.3f%s\n","                           From Monthly Averages = (",yearly_mean/no_of_months,")")
            println("=============================================================================\n")
            
            # reset counters
            yearly_mean = 0
            no_of_months = 0
        end

        # move to start of next month
        start_date = end_date

    end

    println("Total number of Gaps = ",total_gaps,"; Total number of Good values = ", total_good,"\n")
    println(title)
    println("    Year       Mean")
    for i in 1:nrow(yearly_averages_df)
        @printf("%8i %10.3f\n",yearly_averages_df.Year[i],yearly_averages_df.Mean_WL[i])
    end
    flush(stdout)
end

In [ ]:
aa = groupby(transform(msq_df, :Date => x->yearmonthday.(x)),:Date_function)

daily_hi_lo_df = DataFrame(Any[Date[],[],[],[],[]], ["Date", "Low", "Mean", "High", "Range"])
for i in 1:length(aa)
    
    if nrow(aa[i]) .== 144 && length(findall(abs.(aa[i].Reading[:]) .< 9)) .== 144
        
        maxval = maximum(aa[i].Reading)
        meanval = mean((aa[i].Reading))
        minval = minimum(aa[i].Reading)
        range = maxval - minval
    
        push!(daily_hi_lo_df,[Date(aa[i].Date[1]),minval,meanval,maxval,range])
        
    end
end

title = splitext(last(splitdir(infil)))[1]

##outfil = "c:\\users\\waldronj\\Julia programs\\SLR\\Daily_Hi_Lo\\" * title * "_daily_hi_lo_mean.CSV"
outfil = "C:\\Users\\waldronj\\Julia programs\\SLR\\10 Minutes\\21-22\\" * title * "_daily_hi_lo_mean.CSV"
CSV.write(outfil,daily_hi_lo_df)

tm_tick = range(first_date,last_date,step=Year(2))
ticks = Dates.format.(tm_tick,"YY")

a1 = scatter(collect(daily_hi_lo_df.Date),collect(daily_hi_lo_df.High), smooth=:true, lw = 2, color = "red", marker = :circle, markersize = 1.75, markerstrokewidth=1, markerstrokecolor=:red, label="")
a1 = scatter!(collect(daily_hi_lo_df.Date),collect(daily_hi_lo_df.Mean), smooth=:true, lw = 2, color = "blue", marker = :circle, markersize = 1.75, markerstrokewidth=1, markerstrokecolor=:blue, label="")
a1 = scatter!(collect(daily_hi_lo_df.Date),collect(daily_hi_lo_df.Low), smooth=:true, lw = 2, color = "green", marker = :circle, markersize = 1.75, markerstrokewidth=1, markerstrokecolor=:green, label="")

tide_plot = plot(a1, size = (1400, 800), label="",
    
    xtickfontsize=7,ytickfontsize=8, 
    framestyle = :box, legend=:bottomleft, 
    title = title,
    margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

# create a plot file to be saved as a .PNG
plt_file = first(outfil, length(outfil)-4)*"_plot_Hm0.png"

# Save plot to file
savefig(plt_file)
println("Plot file saved as ",plt_file)

bb = groupby(transform(daily_hi_lo_df, :Date => x->year.(x)),:Date_function)
for i in 1:length(bb)
    println(bb[i].Date_function[1],' ',mean(bb[i].Range))
end 

display(tide_plot)


In [ ]:
using Dates

daily = groupby(transform(msq_df, :Date => x->yearmonthday.(x)),:Date_function);
monthly = groupby(transform(msq_df, :Date => x->yearmonth.(x)),:Date_function);
yearly = groupby(transform(msq_df, :Date => x->year.(x)),:Date_function);

for i in 1:length(yearly)
    println(yearly[i].Date_function[1],' ',mean(yearly[i].Reading))
end

println("-------------------------")

for i in 1:length(monthly)
    println(monthly[i].Date_function[1][1],' ',Dates.monthname(monthly[i].Date_function[1][2]),' ',mean(monthly[i].Reading))
    if monthly[i].Date_function[1][2] == 12
        println("")
    end
end   

println("-------------------------")
"""
for i in 1:length(daily)
    println(daily[i].Date_function[1],' ',mean(daily[i].Reading))
end
"""


In [ ]:
aa = combine(groupby(transform(msq_df, :Date => x->yearmonthday.(x)),:Date_function), :Reading => minimum, :Reading => mean, :Reading => maximum, :Reading => length)
date_vals = [getindex.(aa.Date_function,i) for i in eachindex(aa.Date_function[1])]
daily_dates = Date.(date_vals[1],date_vals[2],date_vals[3])

bb = combine(groupby(transform(msq_df, :Date => x->yearmonth.(x)),:Date_function), :Reading => minimum, :Reading => mean, :Reading => maximum, :Reading => length)
date_vals = [getindex.(bb.Date_function,i) for i in eachindex(bb.Date_function[1])]
monthly_dates = Date.(date_vals[1],date_vals[2])

cc = combine(groupby(transform(msq_df, :Date => x->year.(x)),:Date_function), :Reading => minimum, :Reading => mean, :Reading => maximum, :Reading => length)
date_vals = [getindex.(cc.Date_function,i) for i in eachindex(cc.Date_function[1])]
annual_dates = Date.(date_vals[1])

first_date = first(monthly_dates)
last_date = last(monthly_dates)

# display plots to screen
tm_tick = range(first_date,last_date,step=Month(6))
ticks = Dates.format.(tm_tick,"YY-mm")

plt1 = plot(monthly_dates,bb.Reading_mean, markershape=:circle, label="Monthly Mean", smooth=true, color=:blue, markeralpha=.5, lw=2, ms=:3, z_order=:1)
plt1 = plot!(annual_dates,cc.Reading_mean, markershape=:diamond, label="Annual Mean", smooth=true, color=:yellow, markeralpha=:1, markersize=:6, lw=:2)
plt1 = plot!(daily_dates,aa.Reading_maximum, label="Daily Maximum", smooth=true, color=:green, markeralpha=0, lw=:2, alpha=:0.1, fill=(cc.Reading_mean,:green), fillalpha=:0.2)
plt1 = plot!(daily_dates,aa.Reading_minimum, label="Daily Minimum", smooth=true, color=:red, markeralpha=0, lw=:2, alpha=:0.1, fill=(cc.Reading_mean,:red), fillalpha=:0.1)

plot(plt1, lw=:2,
    xlims=(first_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=7,
#    ylims=[0,2],
    ytickfontsize=8,
    size = (1400, 500), 
    framestyle=:box, legend=:topleft, 
    legend_title=title,legend_title_font_halign=:hcenter,legend_title_font_valign=:vcenter,
    fg_legend=:transparent, bg_legend=:transparent,
    margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

In [ ]:
using CSV
using CurveFit
using XLSX
using Dates, DataFrames, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics
using Tk
using GLM
using Polynomials

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

excel_directory = ".\\" 
excel_file = excel_directory*"/Hi_Lo_Mean.xlsx"

work_sheets = XLSX.sheetnames(XLSX.readxlsx(excel_file))

for ii in 1:length(work_sheets)-1
    
    println(ii,' ',work_sheets[ii])
    global df1 = DataFrame(XLSX.readtable(excel_file,ii,))
    df1 = filter(:Date => x -> x >= Date(1986,1,1), df1)
    col_names = names(df1);
    df1.Date = Date.(df1.Date)
    df1.Low = Float64.(df1.Low)
    df1.Mean = Float64.(df1.Mean)
    df1.High = Float64.(df1.High)
    df1.Range = Float64.(df1.Range)

    global aa = combine(groupby(transform(df1, :Date => x->yearmonthday.(x)),:Date_function), :Low => minimum, :Mean => mean, :High => maximum, :Range => length)
    date_vals = [getindex.(aa.Date_function,i) for i in eachindex(aa.Date_function[1])]
    daily_dates = Date.(date_vals[1],date_vals[2],date_vals[3])

    global bb = combine(groupby(transform(df1, :Date => x->yearmonth.(x)),:Date_function), :Low => minimum, :Mean => mean, :High => maximum, :Range => length)
    date_vals = [getindex.(bb.Date_function,i) for i in eachindex(bb.Date_function[1])]
    monthly_dates = Date.(date_vals[1],date_vals[2])

    global cc = combine(groupby(transform(df1, :Date => x->year.(x)),:Date_function), :Low => mean, :Mean => mean, :High => mean)
    date_vals = [getindex.(cc.Date_function,i) for i in eachindex(cc.Date_function[1])]
    annual_dates = Date.(date_vals[1]) + Month(6)
    
    # Calculate the annual rise of Highs in mm/Y
    df = DataFrame(X = [trunc.(Int,(datetime2julian.(DateTime.(string(x[1],'-',x[2],'-',x[3],"T0:0:0"))))) for x in aa.Date_function])
    df.Y = aa.High_maximum
    model1 = lm(@formula(Y ~ X), df)
    aa1 = []
    for i in df.X
        push!(aa1,i*coef(model1)[2]+coef(model1)[1])
    end    
    
    # Calculate the annual rise of Lows in mm/Y
    df = DataFrame(X = [trunc.(Int,(datetime2julian.(DateTime.(string(x[1],'-',x[2],'-',x[3],"T0:0:0"))))) for x in aa.Date_function])
    df.Y = aa.Low_minimum
    model2 = lm(@formula(Y ~ X), df)
    aa2 = []
    for i in df.X
        push!(aa2,i*coef(model2)[2]+coef(model2)[1])
    end        

    # Calculate the annual rise of mean sea level in mm/Y
    df = DataFrame(X = cc.Date_function, Y = cc.Mean_mean)
    model = lm(@formula(Y ~ X), df)
    cc1 = []
    for i in df.X
        push!(cc1,i*coef(model)[2]+coef(model)[1])
    end 
    
        # Calculate the annual rise of daily maximums in mm/Y
    df = DataFrame(X = cc.Date_function, Y = cc.High_mean)
    model2 = lm(@formula(Y ~ X), df)
    cc2 = []
    for i in df.X
        push!(cc2,i*coef(model2)[2]+coef(model2)[1])
    end 

        # Calculate the annual rise of daily minimums in mm/Y
    df = DataFrame(X = cc.Date_function, Y = cc.Low_mean)
    model3 = lm(@formula(Y ~ X), df)
    cc3 = []
    for i in df.X
        push!(cc3,i*coef(model3)[2]+coef(model3)[1])
    end 
    
    dd = combine(groupby(transform(df1, :Date => x->year.(x)),:Date_function), :Low => mean, :High => mean)
    
    first_date = Date(1986,1,1) ##first(monthly_dates)
    last_date = Date(2025,1,1) ##last(monthly_dates)

    # display plots to screen
    tm_tick = range(first_date,last_date,step=Year(2))
    ticks = Dates.format.(tm_tick,"YY")
    
    # Plot the annual mean 
    plot(Date.(cc.Date_function),cc.Mean_mean,lw=3,label="")

    # Plot the annual trends Lo, Mean, Hi
    plt1 = plot!(annual_dates,cc1,markershape=:diamond, lw=1, color=:blue, ls=:dot, ma=:0.75, mc=:blue, ms=:5, label="Mean (Annual trend) = "*string(@sprintf("%0.1f",coef(model)[2]*1000))*" mm/Yr.")
    plt1 = plot!(annual_dates,cc2,markershape=:utriangle,lw=2, color=:green, ma=:0.25, ls=:dash, label="Daily Maximum = "*string(@sprintf("%0.1f",coef(model2)[2]*1000))*" mm/Yr.")
    plt1 = plot!(annual_dates,cc3,markershape=:dtriangle,lw=2, color=:red, ma=:0.25, ls=:dashdot, label="Daily Minimum = "*string(@sprintf("%0.1f",coef(model3)[2]*1000))*" mm/Yr.")

    # Plot daily Hi's and Lo's
    plt1 = plot!(daily_dates,aa.High_maximum, label="", color=:green, lw=:2, alpha=:0.25)
    plt1 = plot!(daily_dates,aa.Low_minimum, label="", color=:red, lw=:2, alpha=:0.25)
    
    xlims = (Date(1985,1,1),Date(2025,1,1))

    tide_plot = plot(plt1, lw=:2,
        xlims=xlims, xticks=(tm_tick,ticks), xtickfontsize=10, tick_direction=:out, tickfontvalign=:bottom,
    #    ylims=[0,2],
        ytickfontsize=12,
        size = (1400, 500), 
        framestyle=:box, legend=:topleft, legend_font_pointsize=:12,
        legend_title=work_sheets[ii]*"                                                                                   ", 
        legend_title_font_halign=:hcenter, legend_title_font_valign=:vcenter,legend_title_font_pointsize=:18, 
        fg_legend=:transparent, bg_legend=:transparent,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)
    
    # create a plot file to be saved as a .PNG
    plt_file = ".\\plots\\"*work_sheets[ii]*"_"*string(year(first(annual_dates))) * "_to_" * string(year(last(annual_dates)))*"_plot.png"

    # Save plot to file
    savefig(plt_file)
    println("Plot file saved as ",plt_file)
    
    display(tide_plot)

        ## Refer https://en.wikipedia.org/wiki/Ordinary_least_squares
    using GLM

    # Calculate the annual rise of mean sea level in mm/Y
    df = DataFrame(X = cc.Date_function, Y = cc.Mean_mean)

    model = lm(@formula(Y ~ X), df)
    cc1 = []
    for i in df.X
        push!(cc1,i*coef(model)[2]+coef(model)[1])
    end

    first_cc = cc[findall(1986 .<= cc.Date_function .<= 2005),:]
    model_first = lm(@formula(Mean_mean ~ Date_function), first_cc)
    first_cc1 = []
    for i in first_cc.Date_function
        push!(first_cc1,i*coef(model)[2]+coef(model)[1])
    end

    middle_cc = cc[findall(1995 .<= cc.Date_function .<= 2014),:]
    model_middle = lm(@formula(Mean_mean ~ Date_function), middle_cc)
    middle_cc1 = []
    for i in middle_cc.Date_function
        push!(middle_cc1,i*coef(model)[2]+coef(model)[1])
    end
    
    last_cc = cc[findall(2003 .<= cc.Date_function),:]
    model_last = lm(@formula(Mean_mean ~ Date_function), last_cc)
    last_cc1 = []
    for i in last_cc.Date_function
        push!(last_cc1,i*coef(model)[2]+coef(model)[1])
    end

    p1 = scatter(Date.(df.X),df.Y,smooth=true, markerstrokewidth=0.5, color=:yellow, lc=:yellow, lw=:4, label="Annual means")
    p1 = plot!(Date.(df.X),predict(model), lw=:2, color=:red, label="Annual trend 1986-2022")
    p1 = plot!(Date.(first_cc.Date_function),predict(model_first), lw=2, ls=:dashdot, label="Annual trend 1986-2005")
    p1 = plot!(Date.(middle_cc.Date_function),predict(model_middle), lw=2, ls=:dashdotdot, label="Annual trend 1995-2014")    
    p1 = plot!(Date.(last_cc.Date_function),predict(model_last), lw=2, ls=:dot, label="Annual trend 2003-2022")
    
    fit = curve_fit(Polynomial, df.X, df.Y, 2)
    y0b = fit.(df.X) 
##    p1 = plot!(Date.(df.X), y0b, lw=:3, color=:black, ls=:dash, label="2nd-order Polynomial fit", z_order=:1)

    first_date = Date(1985) ##first(monthly_dates)
    last_date = Date(2025) ##last(monthly_dates)

    # display plots to screen
    tm_tick = range(first_date,last_date,step=Year(2))
    ticks = Dates.format.(tm_tick,"YY")

    p1_plot = plot(p1,
        xlims=(first_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=10, tick_direction=:out, tickfontvalign=:bottom,
        #    ylims=[0,2],
        ytickfontsize=12,
        size = (1400, 500), 
        framestyle=:box, legend=:bottomright, legend_font_pointsize=:12,
        legend_title=work_sheets[ii], 
        legend_title_font_halign=:hcenter, legend_title_font_valign=:vcenter,legend_title_font_pointsize=:18, 
        fg_legend=:transparent, bg_legend=:transparent,
        margin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

    # create a plot file to be saved as a .PNG
    plt_file = ".\\plots\\"*work_sheets[ii]*"_"*string(year(first(annual_dates))) * "_to_" * string(year(last(annual_dates)))*"_two_trends_plot.png"

    # Save plot to file
    savefig(plt_file)
    println("Plot file saved as ",plt_file)

    display(p1_plot)
    
end

## Calculate annual rates of SLR

In [ ]:
using CSV
using CurveFit
using XLSX
using Dates, DataFrames, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics
using Tk
using GLM
using Polynomials

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

excel_directory = ".\\" 
excel_file = excel_directory*"/Hi_Lo_Mean.xlsx"

work_sheets = XLSX.sheetnames(XLSX.readxlsx(excel_file))

for jj in 1:2 #length(work_sheets)-1
    
    println(jj,' ',work_sheets[jj])
    global df1 = DataFrame(XLSX.readtable(excel_file,jj,))
    df1 = filter(:Date => x -> x >= Date(1985,1,1), df1)
    col_names = names(df1);
    df1.Date = Date.(df1.Date)
    df1.Low = Float64.(df1.Low)
    df1.Mean = Float64.(df1.Mean)
    df1.High = Float64.(df1.High)
    df1.Range = Float64.(df1.Range)

    yearly_df = groupby(transform(df1, :Date => x->year.(x)),:Date_function)
    year_diff = last(yearly_df[1].Mean)-first(yearly_df[1].Mean)

    #first(yearly_df[1].Date_function)
    
    old_mean = mean(yearly_df[1].Mean)
    
    counts = unique(groupindices(yearly_df))
    
    for ii in counts[2:last(counts)]
        
        @printf("%4d %6.3f\n",first(yearly_df[ii].Date_function),mean(yearly_df[ii].Mean)-old_mean)
        
        old_mean = mean(yearly_df[ii].Mean)

    end
    
end

## Applying piecewise regression

In [ ]:
using GLM, StatsPlots 

function add_breakpoint(data, bp)
    data[!, "since_bp"] = max.(0, data[!, "Date_function"] .- bp);
end;


function fit_piecewise(data, minbp, maxbp)
    min_deviance = Inf
    best_model = nothing
    best_bp = 0
    current_model = nothing

    for bp in minbp:maxbp
        add_breakpoint(data, bp)
        current_model = lm(@formula(Mean_mean ~ Date_function + since_bp), data)
        if deviance(current_model) < min_deviance
            min_deviance = deviance(current_model)
            best_model = current_model
            best_bp = bp
        end
    end

    return best_model, best_bp
end;


# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

excel_directory = ".\\" 
excel_file = excel_directory*"/Hi_Lo_Mean.xlsx"

work_sheets = XLSX.sheetnames(XLSX.readxlsx(excel_file))

for ii in 1:length(work_sheets)-1
    
    println(ii,' ',work_sheets[ii])
    global df1 = DataFrame(XLSX.readtable(excel_file,ii,))
    df1 = filter(:Date => x -> x >= Date(1986,1,1), df1)
    col_names = names(df1);
    df1.Date = Date.(df1.Date)
    df1.Low = Float64.(df1.Low)
    df1.Mean = Float64.(df1.Mean)
    df1.High = Float64.(df1.High)
    df1.Range = Float64.(df1.Range)    
    

    global cc = combine(groupby(transform(df1, :Date => x->year.(x)),:Date_function), :Low => mean, :Mean => mean, :High => mean);
    date_vals = [getindex.(cc.Date_function,i) for i in eachindex(cc.Date_function[1])]
    annual_dates = Date.(date_vals[1])

    global lm2,best_bp = fit_piecewise(cc, 2000, 2013)
    add_breakpoint(cc, best_bp)

    lm1 = lm(@formula(Mean_mean ~ Date_function), cc)

    cc[!, "prediction"] = predict(lm2);
    predictions = predict(lm2, cc; interval = :confidence, level = 0.95);

    cc[!, "prediction"] = predictions[!, "prediction"];
    cc[!, "lower"] = predictions[!, "lower"];
    cc[!, "upper"] = predictions[!, "upper"];

    p1 = scatter(Date.(cc.Date_function),cc.Mean_mean,smooth=false, markerstrokewidth=0.5, color=:blue, lc=:yellow, lw=:5, label="Annual means")
    p1 = plot!(Date.(cc.Date_function), cc.prediction, fillrange=[cc.upper cc.prediction], fillalpha=0.05, c=:blue, label="95% confidence limits")
    p1 = plot!(Date.(cc.Date_function), cc.prediction, fillrange=[cc.lower cc.prediction], fillalpha=0.05, c=:blue, label="")    

    first_date = Date(1985) ##first(monthly_dates)
    last_date = Date(2025) ##last(monthly_dates)

    # display plots to screen
    tm_tick = range(first_date,last_date,step=Year(2))
    ticks = Dates.format.(tm_tick,"YY")

    p1_plot = plot(p1,
        xlims=(first_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=10, tick_direction=:out, tickfontvalign=:bottom,
        #    ylims=[0,2],
        ytickfontsize=12,
        size = (1400, 500), 
        framestyle=:box, legend=:bottomright, legend_font_pointsize=:12,
        legend_title=work_sheets[ii], 
        legend_title_font_halign=:hcenter, legend_title_font_valign=:vcenter,legend_title_font_pointsize=:18, 
        fg_legend=:transparent, bg_legend=:transparent,
        margin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)
    
    # create a plot file to be saved as a .PNG
    plt_file = ".\\plots\\"*work_sheets[ii]*"_"*string(year(first(annual_dates))) * "_to_" * string(year(last(annual_dates)))*"_piecewise_regression_plot.png"

    # Save plot to file
    savefig(plt_file)
    println("Plot file saved as ",plt_file)

    display(p1_plot)
    
end

## Calculate t and P values

In [ ]:
## Refer https://en.wikipedia.org/wiki/Ordinary_least_squares
## Also https://scholarworks.wm.edu/reports/2804/

## Citation: Boon, J. D. (2007) Sea Coast and Sea Level Trends. Manuscript. Virginia Institute of Marine Science, William & Mary. https://scholarworks.wm.edu/reports/2804
##########################################################################################################################################################################
using GLM

# Calculate the annual rise of mean sea level in mm/Y
df = DataFrame(X = cc.Date_function, Y = cc.Mean_mean)

model = lm(@formula(Y ~ X), df)
cc1 = []
for i in df.X
    push!(cc1,i*coef(model)[2]+coef(model)[1])
end

## Least Squares linear regression 
X = Float64.(df.X .- df.X[1])
Y = df.Y

XY = X.*Y
X² = X.^2

x̄ = mean(X)
ȳ = mean(Y)

∑X = sum(X)
∑Y = sum(Y)
∑XY = sum(XY)
∑X² = sum(X²)

n = nrow(df)

b1 = (n * ∑XY - (∑X * ∑Y)) / ((n * ∑X²) - (∑X)^2)    # Slope

a = (∑Y - (b * ∑X))/n    # Intercept

x̄ = ∑X/n
ȳ = ∑Y/n
x = X.-x̄
y = Y.-ȳ

∑xy = sum(x.*y)
∑x² = sum(x.^2)
∑y² = sum(y.^2)

m = ∑xy/∑x²    # Regression coefficient
b = ȳ - m*x̄    # Y-axis intercept

∑d² = ∑y² - ∑xy^2/∑x²    # sum of squares of deviation from regression
s² = ∑d² / (n-2)    # mean square deviation from regression
s = √s²
sₘ = s / √∑x²    # sample standard error of the regression coefficient

Xₜ = m/sₘ    # X-value t-statistic

@printf("Coef. = %5.8f  Std.Error = %5.9f  t = %5.2f\n", m, sₘ, Xₜ)
@printf("Deviance = %5.8f\n", ∑d²)

println(model)

In [ ]:
df = DataFrame(X = cc.Date_function, Y = cc.Mean_mean)
df2 = DataFrame(X = Dates.datetime2julian.(DateTime.(df1.Date)) .- Dates.datetime2julian.(DateTime.(1985)), Y = df1.Mean)
lm1 = lm(@formula(Y ~ X), df)
lm2 = lm(@formula(Y ~ X), df2)

println(lm1)

println(lm2)

In [ ]:
global xx = combine(groupby(transform(msq_df, :Date => x->year.(x)),:Date_function), :Reading => mean);
date_vals = [getindex.(xx.Date_function,i) for i in eachindex(xx.Date_function[1])]
annual_dates = Date.(date_vals[1])

## Read Excel file containing PDO and SOI details

In [ ]:
# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

excel_directory = ".\\" 
excel_file = excel_directory*"/PDO_SOI.xlsx"

work_sheets = XLSX.sheetnames(XLSX.readxlsx(excel_file))

global pdo = DataFrame(XLSX.readtable(excel_file,1,))
global soi = DataFrame(XLSX.readtable(excel_file,2,))
global sam = DataFrame(XLSX.readtable(excel_file,3,))
global iod = DataFrame(XLSX.readtable(excel_file,4,))
global enso = DataFrame(XLSX.readtable(excel_file,5,))

soi.Date = Date.(soi.Year,soi.Month) 

monthly_pdo = DataFrame([Date[],Real[]], ["Date", "PDO",])
for i in 1:nrow(pdo)
    for j in 2:13
        push!(monthly_pdo,[Date.(pdo.Year[i],j-1),pdo[i,j]])
    end
end

monthly_sam = DataFrame([Date[],Real[]], ["Date", "SAM",])
for i in 1:nrow(sam)
    for j in 2:13
        push!(monthly_sam,[Date.(sam.Year[i],j-1),sam[i,j]])
    end
end

monthly_iod = DataFrame([Date[],Real[]], ["Date", "IOD",])
for i in 1:nrow(iod)
    for j in 2:13
        push!(monthly_iod,[Date.(iod.Year[i],j-1),iod[i,j]])
    end
end

monthly_enso = DataFrame([Date[],Real[]], ["Date", "ENSO",])
for i in 1:nrow(enso)
    for j in 2:13
        push!(monthly_enso,[Date.(enso.Year[i],j-1),enso[i,j]])
    end
end
first_date = Date(1985) ##first(monthly_dates)
last_date = Date(2025) ##last(monthly_dates)

# display plots to screen
tm_tick = range(first_date,last_date,step=Year(2))
ticks = Dates.format.(tm_tick,"YY")

title = plot(title = "PDO and SOI", grid = false, showaxis = false, bottom_margin = -60Plots.px)

p1 = plot(monthly_pdo.Date,monthly_pdo.PDO, lw=:2, color=:red, smooth=true, grid=true, label="Pacific Decadal Oscillation", legend=:topright)

p2 = plot(soi.Date,soi.SOI, lw=:2, ls = :dot, color=:green, smooth=true, grid=true, label="Southern Oscillation Index",legend=:topright)
p3 = plot(monthly_sam.Date,monthly_sam.SAM, lw=:2, color=:blue, smooth=true, grid=true, label="Southern Annular Mode", legend=:topright)
p4 = plot(monthly_iod.Date,monthly_iod.IOD, lw=:2, color=:orange, smooth=true, grid=true, label="Indian Ocean Dipole", legend=:topright)
p5 = plot(monthly_enso.Date,monthly_enso.ENSO, lw=:2, color=:pink, smooth=true, grid=true, label="ENSO", legend=:topright)

plot_p1 = plot(title, p1, p2, p3, p4, p5, layout = @layout([A{0.01h}; Plots.grid(5,1)]), xlims=(first_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=10, tick_direction=:out, tickfontvalign=:bottom,
        ytickfontsize=12,
        size = (1400, 1000), 
        framestyle=:box, legend_font_pointsize=:12, 
        fg_legend=:transparent, bg_legend=:transparent,
        leftmargin = 15Plots.mm, rightmargin = 15Plots.mm, topmargin=5Plots.mm, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

# create a plot file to be saved as a .PNG
plt_file = ".\\plots\\PDO_SOI.png"

# Save plot to file
savefig(plt_file)
println("Plot file saved as ",plt_file)

display(plot_p1)

## Plot STFT of PDO

In [ ]:
using DSP, ContinuousWavelets, Plots, Wavelets, FFTW

gr()

nw=64;
spec = DSP.Periodograms.spectrogram(monthly_pdo.PDO, nw, 60; fs=1,window=hanning);
power_spec = DSP.Periodograms.power(spec)
max_spec = maximum(power_spec)

start_date = Date(1986) #first(monthly_pdo.Date)
last_date = Date(2022) #last(monthly_pdo.Date)

# display plots to screen
tm_tick = range(Date(1985),Date(2025),step=Year(2))
ticks = Dates.format.(tm_tick,"YY")

p1 = contourf(first(monthly_pdo.Date) + Month.(ceil.((spec.time))), spec.freq, DSP.Periodograms.power(spec), lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)

# draw grid lines on plot
for i in 0:0.01:0.1
    hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i in start_date:Year(2):last_date
    vline!(p1, [i], lw=0.5, c=:white, label="")
end

p1_plot = plot(p1, xlabel="Year", xlim=(xlim=(Date(1985),Date(2025))), xticks=(tm_tick,ticks), xtickfontsize=7,
        ylabel="Frequency (Hz)", ylim=(0,0.1), ytickfontsize=8, 
        title="Pacific Decadal Oscillation", framestyle = :box,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1600,500), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=false)

# create a plot file to be saved as a .PNG
plt_file = ".\\plots\\PDO.png"

# Save plot to file
savefig(plt_file)
println("Plot file saved as ",plt_file)

display(p1_plot)

## Plot STFT of SOI

In [ ]:
gr()

nw=64;
spec = DSP.Periodograms.spectrogram(Float64.(soi.SOI), nw, 60; fs=1, window=hanning);
power_spec = DSP.Periodograms.power(spec)
max_spec = maximum(power_spec)

start_date = Date(1985) #first(soi.Date)
last_date = Date(2025) #last(soi.Date)

# display plots to screen
tm_tick = range(start_date,last_date,step=Year(2))
ticks = Dates.format.(tm_tick,"YY")

p1 = contourf(first(soi.Date) + Month.(ceil.((spec.time))), spec.freq, DSP.Periodograms.power(spec), lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)

# draw grid lines on plot
for i in 0:0.01:0.1
    hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i in start_date:Year(2):last_date
    vline!(p1, [i], lw=0.5, c=:white, label="")
end

p1_plot = plot(p1, xlabel="Year", xlim=(Date(1985),Date(2025)), xticks=(tm_tick,ticks), xtickfontsize=7,
        ylabel="Frequency (Hz)", ylim=(0,0.1), ytickfontsize=8, 
        title="Southern Oscillation Index", framestyle = :box,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1600,500), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=false)

# create a plot file to be saved as a .PNG
plt_file = ".\\plots\\SOI.png"

# Save plot to file
savefig(plt_file)
println("Plot file saved as ",plt_file)

display(p1_plot)

## Plot STFT of SAM

In [ ]:
gr()

nw=64;
spec = DSP.Periodograms.spectrogram(Float64.(monthly_sam.SAM), nw, 60; fs=1, window=hanning);
power_spec = DSP.Periodograms.power(spec)
max_spec = maximum(power_spec)

start_date = Date(1985) #first(soi.Date)
last_date = Date(2025) #last(soi.Date)

# display plots to screen
tm_tick = range(start_date,last_date,step=Year(2))
ticks = Dates.format.(tm_tick,"YY")

p1 = contourf(first(monthly_sam.Date) + Month.(ceil.((spec.time))), spec.freq, DSP.Periodograms.power(spec), lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)

# draw grid lines on plot
for i in 0:0.01:0.1
    hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i in start_date:Year(2):last_date
    vline!(p1, [i], lw=0.5, c=:white, label="")
end

p1_plot = plot(p1, xlabel="Year", xlim=(Date(1985),Date(2025)), xticks=(tm_tick,ticks), xtickfontsize=7,
        ylabel="Frequency (Hz)", ylim=(0,0.1), ytickfontsize=8, 
        title="Southern Annual Mode", framestyle = :box,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1600,500), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=false)

# create a plot file to be saved as a .PNG
plt_file = ".\\plots\\SAM.png"

# Save plot to file
savefig(plt_file)
println("Plot file saved as ",plt_file)

display(p1_plot)

## Plot STFT of IOD

In [ ]:
gr()

nw=64;
spec = DSP.Periodograms.spectrogram(Float64.(monthly_iod.IOD), nw, 60; fs=1, window=hanning);
power_spec = DSP.Periodograms.power(spec)
max_spec = maximum(power_spec)

start_date = Date(1985) #first(soi.Date)
last_date = Date(2025) #last(soi.Date)

# display plots to screen
tm_tick = range(start_date,last_date,step=Year(2))
ticks = Dates.format.(tm_tick,"YY")

p1 = contourf(first(monthly_iod.Date) + Month.(ceil.((spec.time))), spec.freq, DSP.Periodograms.power(spec), lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)

# draw grid lines on plot
for i in 0:0.01:0.1
    hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i in start_date:Year(2):last_date
    vline!(p1, [i], lw=0.5, c=:white, label="")
end

p1_plot = plot(p1, xlabel="Year", xlim=(Date(1985),Date(2025)), xticks=(tm_tick,ticks), xtickfontsize=7,
        ylabel="Frequency (Hz)", ylim=(0,0.1), ytickfontsize=8, 
        title="Indian Ocean Dipole", framestyle = :box,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1600,500), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=false)

# create a plot file to be saved as a .PNG
plt_file = ".\\plots\\IOD.png"

# Save plot to file
savefig(plt_file)
println("Plot file saved as ",plt_file)

display(p1_plot)

In [ ]:
gr()

nw=64;
spec = DSP.Periodograms.spectrogram(Float64.(monthly_enso.ENSO), nw, 60; fs=1, window=hanning);
power_spec = DSP.Periodograms.power(spec)
max_spec = maximum(power_spec)

start_date = Date(1985) #first(soi.Date)
last_date = Date(2025) #last(soi.Date)

# display plots to screen
tm_tick = range(start_date,last_date,step=Year(2))
ticks = Dates.format.(tm_tick,"YY")

years = []
append!(years,100.)
for i in 2:length(spec.freq)
    append!(years,1/DSP.Periodograms.freq(spec)[:,1][i])
end
p1 = contourf(first(monthly_enso.Date) + Month.(ceil.((spec.time))), reverse(years), DSP.Periodograms.power(spec), yflip=:true, lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)

# draw grid lines on plot
for i in 100:-10:0
    hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i in start_date:Year(2):last_date
    vline!(p1, [i], lw=0.5, c=:white, label="")
end

p1_plot = plot(p1, xlabel="Year", xlim=(Date(1985),Date(2025)), xticks=(tm_tick,ticks), xtickfontsize=7,
        ylabel="Frequency (Hz)", ytickfontsize=8, 
        title="ENSO", framestyle = :box,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1600,500), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=false)

# create a plot file to be saved as a .PNG
plt_file = ".\\plots\\enso.png"

# Save plot to file
savefig(plt_file)
println("Plot file saved as ",plt_file)

display(p1_plot)

In [ ]:
years = []
append!(years,100.)
for i in 2:length(spec.freq)
    append!(years,1/DSP.Periodograms.freq(spec)[:,1][i])
end

In [ ]:
using Distributions

function simulate_outcome(null_model)
  resid_distr = Normal(0, dispersion(null_model.model))
  prediction = fitted(null_model)
  new_outcome = prediction + rand(resid_distr, length(prediction))
  return new_outcome
end;


function one_run(data, null_model, min_bp, max_bp)
  new_outcome = simulate_outcome(null_model)
  data[!, "GJT"] = new_outcome
  best_model = fit_piecewise(data, min_bp, max_bp)
  pval = coeftable(best_model[1]).cols[4][3]
  return pval
end;


function generate_p_distr(data, null_model, min_bp, max_bp, n_runs)
  pvals = [one_run(data, null_model, min_bp, max_bp) for _ in 1:n_runs]
  return pvals
end;


n_runs = 25_000;
pvals = generate_p_distr(cc, lm1, 2005, 2015, n_runs);

sum(pvals .<= 0.0472) / n_runs